In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist

In [2]:
(x_trn, y_trn), (x_tst, y_tst) = mnist.load_data()

In [5]:
num_features = 28 * 28
num_classes = 10

lr = 0.01
epochs = 1000
batch_size = 32
display_epoch = 100

In [6]:
x_train = (np.array(x_trn, np.float32)).reshape([-1, num_features]) / 255.0
x_test = (np.array(x_tst, np.float32)).reshape([-1, num_features]) / 255.0

y_train = tf.one_hot(y_trn, depth = num_classes)
y_test = tf.one_hot(y_tst, depth = num_classes)

In [8]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)